In [2]:
import requests
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import batch_process, log_process, authenticate, fetch_items_from_storage
from src.config import settings
import json
import asyncio
import aiohttp
from datetime import datetime

In [3]:
json = {
  "access_token": None,
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)

Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json


In [4]:
access_token = authenticate(json['client_id'], json['client_secret'])

In [ ]:
bigquery.run_query()

In [39]:
catalog_item_id = 'MLB28887155'

url = f"https://api.mercadolibre.com/products/{catalog_item_id}/items"

# Cabeçalhos de autorização
headers = {'Authorization': f'Bearer {access_token}'}

# Fazendo a requisição GET
response = requests.get(url, headers=headers)
response.json()

{'paging': {'total': 8, 'offset': 0, 'limit': 100},
 'results': [{'item_id': 'MLB4966133390',
   'site_id': 'MLB',
   'seller_id': 1831599929,
   'accepts_mercadopago': True,
   'price': 76.9,
   'category_id': 'MLB269928',
   'currency_id': 'BRL',
   'warranty': 'Garantia de fábrica: 12 meses',
   'condition': 'new',
   'listing_type_id': 'gold_special',
   'international_delivery_mode': 'none',
   'tier': '',
   'inventory_id': '',
   'tags': ['kvs_primary',
    'good_quality_thumbnail',
    'extended_warranty_eligible',
    'immediate_payment',
    'cart_eligible'],
   'deal_ids': [],
   'official_store_id': None,
   'original_price': None,
   'shipping': {'free_shipping': False,
    'store_pick_up': False,
    'mode': 'me2',
    'logistic_type': 'drop_off',
    'tags': []},
   'seller_address': {'city': {'id': 'TUxCQ1NQLTIzNTE', 'name': 'São Paulo'},
    'state': {'id': 'TUxCUFNBT085N2E4', 'name': 'São Paulo'},
    'neighborhood': {'id': 'TUxCQlZJTGwzZ3N0', 'name': 'Vila Ede'}},
  

In [5]:
item_id = 'MLB4966133390'

url = f"https://api.mercadolibre.com/items/{item_id}/shipping"

# Cabeçalhos de autorização
headers = {'Authorization': f'Bearer {access_token}'}

# Fazendo a requisição GET
response = requests.get(url, headers=headers)
response.json()

{'channels': [{'id': 'marketplace',
   'mode': 'me2',
   'logistic_type': 'drop_off',
   'methods': [],
   'tags': [],
   'dimensions': None,
   'local_pick_up': False,
   'free_shipping': False,
   'store_pick_up': False}],
 'default': 'marketplace'}

In [8]:
item_id = 'MLB4978023790'

url = f"https://api.mercadolibre.com/items/{item_id}/shipping"

# Cabeçalhos de autorização
headers = {'Authorization': f'Bearer {access_token}'}

# Fazendo a requisição GET
response = requests.get(url, headers=headers)
data = response.json()

In [11]:
def process_shipping(json_data):
    try:
        default_value = json_data.get('default')
        channels = json_data.get('channels', [])
        item_id = json_data.get('item_id')
        dict_list = []
        for channel in channels:
            dict_content = {
                'item_id': item_id,
                'channel_id': channel.get('id'),
                'mode': channel.get('mode'),
                'logistic_type': channel.get('logistic_type'),
                'local_pick_up': channel.get('local_pick_up'),
                'free_shipping': channel.get('free_shipping'),
                'store_pick_up': channel.get('store_pick_up'),
                'default': default_value
            }
            dict_list.append(dict_content)
        return dict_list
    except Exception as e:
        print(f'Error processing json: {json_data}, error: {e}')
        return []
import pandas as pd
pd.DataFrame(process_shipping(data))

,channel_id,mode,logistic_type,local_pick_up,free_shipping,store_pick_up,default
0,marketplace,me2,fulfillment,False,True,False,marketplace
1,mshops,me2,fulfillment,False,False,False,marketplace


In [24]:
data = json
client_id = data.get('client_id')
client_secret = data.get('client_secret')
store_name = data.get('store_name')
seller_id = data.get('seller_id')
access_token = data.get('access_token')
print('** Defining authentication... **')
# Authenticate (assuming this is now centralized in utils.py or a similar file)
if not access_token:
    access_token = authenticate(client_id, client_secret)  # You can add this to a common module
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_FULLFILMENT
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')

# Fetch item IDs from the storage bucket
blob_items_prefix = f'{store_name}/meli/api_response/catelog_details/date={today_str}/'
items_id = fetch_items_from_storage(
storage, 
bucket_name, 
blob_items_prefix, 
key_names='inventory_id'
)

** Defining authentication... **
** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-10-07/batch_0__process_time=2024-10-07T07:00:00.333629-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-10-07/batch_1__process_time=2024-10-07T07:00:00.614209-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-10-07/batch_2__process_time=2024-10-07T07:00:00.954918-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-10-07/batch_3__process_time=2024-10-07T07:00:00.215978-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-10-07/batch_4__process_time=2024-10-07T07:00:00.670108-03:00.json
Reading file: hubsmarthome/meli/

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.config import settings
import json


def insert_bq_competitors_prices(request):

    data = request.get_json()
    store_name = data.get('store_name')
    seller_id = data.get('seller_id')

    print('** Connecting to storage and BigQuery... **')
    # Initialize storage and BigQuery
    storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
    bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)

    # Define paths and table names from the config
    bucket_name = settings.BUCKET_STORES
    table_management = settings.TABLE_MANAGEMENT
    destiny_table = settings.TABLE_CATALOG_COMPETITORS
    blob_shipping_cost = settings.BLOB_COMPETITORS_CATALOG(store_name)

    # Define today's date
    today_str = datetime.today().strftime('%Y-%m-%d')

    # Get dates to treat
    list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)

    print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process  ***')

    df_processed_data = pd.DataFrame()

    for date in list_dates_to_process:

        # Transform date to string
        date_to_process = date.strftime('%Y-%m-%d')
        print(f'Processing date: {date_to_process}')
        # Get blob with the date
        blob_prefix = blob_shipping_cost + f'date={date_to_process}/'
        # List all the files
        blobs = storage.list_blobs(bucket_name, blob_prefix)

        # Processing each blob
        for blob in blobs:
            print(f"Reading file: {blob.name}")
            content = storage.download_json(bucket_name, blob.name)

            for json in content:
                processed_dict = process_prices(json)

                if isinstance(processed_dict, list):
                    df_processed_data = pd.concat([df_processed_data, pd.DataFrame(processed_dict)], ignore_index = True)
                else:
                    continue

        df_processed_data['correspondent_date'] = pd.to_datetime(date_to_process)
        df_processed_data['process_time'] = datetime.now()
        df_processed_data['seller_id'] = seller_id

        print(f'*** Finished treating all data. {df_processed_data.shape[0]} products ***')

        print('** Deleting existing data **')
        bigquery.delete_existing_data(destiny_table, seller_id, date_to_process)
        
        print('** Correct dataframe schema **')
        bigquery.match_dataframe_schema(df_processed_data, destiny_table)

        print('** Inserting data into BQ**')
        bigquery.insert_dataframe(df_processed_data, destiny_table)

        print('** Updating log table **')
        bigquery.update_logs_table(seller_id, date_to_process, destiny_table, table_management)

    return ('Success', 200)


def process_prices(json):

    try:
        extracted_data = []
        # Dicionário temporário para priorizar os preços por canal
        price_by_channel = {}
        for price in json['prices']:
            channel = price['conditions']['context_restrictions']
            if len(channel) == 1:
                channel = channel[0]

                # Se ainda não há preço para o canal ou se o preço atual é promoção, atualiza
                if channel not in price_by_channel or price['type'] == 'promotion':
                    price_by_channel[channel] = {
                        'item_id': json.get('id'),
                        'price_id': price.get('id'),
                        'regular_amount': price.get('regular_amount'),
                        'price': price.get('amount'),
                        'channel': channel,
                        'last_updated': price.get('last_updated')
                    }
        # Converte os valores armazenados para uma lista
        extracted_data.extend(price_by_channel.values())

        return extracted_data
    
    except:
        print(f'Error processing json: {json}')
        

In [2]:
json = {
  "access_token": None,
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

In [29]:
data = json
store_name = data.get('store_name')
seller_id = data.get('seller_id')
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_CATALOG_COMPETITORS
blob_shipping_cost = settings.BLOB_COMPETITORS_CATALOG(store_name)
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')
# Get dates to treat
list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)
print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process  ***')
df_processed_data = pd.DataFrame()
for date in list_dates_to_process:
    # Transform date to string
    date_to_process = date.strftime('%Y-%m-%d')
    print(f'Processing date: {date_to_process}')
    # Get blob with the date
    blob_prefix = blob_shipping_cost + f'date={date_to_process}/'
    # List all the files
    blobs = storage.list_blobs(bucket_name, blob_prefix)
    # Processing each blob
    for blob in blobs:
        print(f"Reading file: {blob.name}")
        content = storage.download_json(bucket_name, blob.name)
        for json in content:
            processed_dict = process_competitors_catalog(json)
            if isinstance(processed_dict, list):
                df_processed_data = pd.concat([df_processed_data, pd.DataFrame(processed_dict)], ignore_index = True)
            else:
                continue
    df_processed_data['correspondent_date'] = pd.to_datetime(date_to_process)
    df_processed_data['process_time'] = datetime.now()
    df_processed_data['seller_id'] = seller_id

** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
*** Starting to process dates: 1 dates to process  ***
Processing date: 2024-10-12
Reading file: hubsmarthome/meli/api_response/item_competitors_catalog/date=2024-10-12/batch_0__process_time=2024-10-12T13:09:09.028454-03:00.json
Reading file: hubsmarthome/meli/api_response/item_competitors_catalog/date=2024-10-12/batch_1__process_time=2024-10-12T13:09:09.360037-03:00.json


In [30]:
df_processed_data

,catalog_product_id,item_id,competitors_type,category_id,official_store_id,competitor_seller_id,listing_type_id,condition,correspondent_date,process_time,seller_id
0,MLB26044044,MLB3775563827,catalog,MLB7073,20.0,1168726042,gold_special,new,2024-10-12,2024-10-12 13:33:39.181979,189643563
1,MLB26044044,MLB3674694233,catalog,MLB7073,5637.0,189643563,gold_special,new,2024-10-12,2024-10-12 13:33:39.181979,189643563
2,MLB26044044,MLB3427682547,catalog,MLB7073,1811.0,840561163,gold_special,new,2024-10-12,2024-10-12 13:33:39.181979,189643563
3,MLB26044044,MLB3960321858,catalog,MLB7073,56642.0,663212996,gold_special,new,2024-10-12,2024-10-12 13:33:39.181979,189643563
4,MLB26044044,MLB3960109120,catalog,MLB7073,56642.0,663212996,gold_pro,new,2024-10-12,2024-10-12 13:33:39.181979,189643563
...,...,...,...,...,...,...,...,...,...,...,...
1877,MLB36961829,MLB3757157759,catalog,MLB269930,5637.0,189643563,gold_special,new,2024-10-12,2024-10-12 13:33:39.181979,189643563
1878,MLB36961829,MLB4702995110,catalog,MLB269930,NaN,36218812,gold_special,new,2024-10-12,2024-10-12 13:33:39.181979,189643563
1879,MLB36961829,MLB5071171310,catalog,MLB269930,NaN,150332823,gold_special,new,2024-10-12,2024-10-12 13:33:39.181979,189643563
1880,MLB36961829,MLB5071185100,catalog,MLB269930,NaN,1019724304,gold_special,new,2024-10-12,2024-10-12 13:33:39.181979,189643563


In [5]:
content[0]['item_id']

'MLB22723618'

In [8]:
data = content[0]['results']
data[0]

{'item_id': 'MLB3634176718',
 'site_id': 'MLB',
 'seller_id': 776488593,
 'accepts_mercadopago': True,
 'price': 182.04,
 'category_id': 'MLB7070',
 'currency_id': 'BRL',
 'warranty': 'Garantia de fábrica: 365 dias',
 'condition': 'new',
 'listing_type_id': 'gold_special',
 'international_delivery_mode': 'none',
 'tier': '',
 'inventory_id': '',
 'tags': ['extended_warranty_eligible',
  'kvs_primary',
  'good_quality_thumbnail',
  'immediate_payment',
  'cart_eligible'],
 'deal_ids': [],
 'official_store_id': None,
 'original_price': None,
 'shipping': {'free_shipping': True,
  'store_pick_up': False,
  'mode': 'me2',
  'logistic_type': 'cross_docking',
  'tags': ['mandatory_free_shipping']},
 'seller_address': {'city': {'id': 'TVhYU8OjbyBKb3PDqSBkb3MgQ2FtcG9zVFV4Q',
   'name': 'São José dos Campos'},
  'state': {'id': 'TUxCUFNBT085N2E4', 'name': 'São Paulo'},
  'neighborhood': {'id': 'TVhYSmFyZGltIE5vdmEgQW3DqXJpY2FUVmhZV0',
   'name': 'Jardim Nova América'}},
 'sale_terms': [{'value_

In [21]:
content[0]['results'][0].get('category_id')

'MLB7070'

In [27]:
list_proc = process_competitors_catalog(content[0])
pd.DataFrame(list_proc)

,catalog_id,item_id,competitors_type,category_id,official_store_id,competitor_seller_id,listing_type_id,condition
0,MLB22723618,MLB3634176718,catalog,MLB7070,NaN,776488593,gold_special,new
1,MLB22723618,MLB3307356743,catalog,MLB7070,NaN,38786499,gold_special,new
2,MLB22723618,MLB3355029315,catalog,MLB7070,NaN,795287937,gold_special,new
3,MLB22723618,MLB3355085767,catalog,MLB7070,NaN,795287937,gold_pro,new
4,MLB22723618,MLB3307368279,catalog,MLB7070,NaN,38786499,gold_pro,new
5,MLB22723618,MLB3634325978,catalog,MLB7070,NaN,776488593,gold_pro,new
6,MLB22723618,MLB3313240905,catalog,MLB7070,NaN,713209624,gold_pro,new
7,MLB22723618,MLB4343848374,catalog,MLB7070,874.0,185638169,gold_special,new
8,MLB22723618,MLB4305089272,catalog,MLB7070,5637.0,189643563,gold_special,new
9,MLB22723618,MLB3955400158,catalog,MLB7070,NaN,22606787,gold_special,new


In [28]:
def process_competitors_catalog(json):

    catalog_id = json['item_id']
    results_list = []  # Create an empty list to store the dictionaries

    try:
        for item in json['results']:
            dict_content = {
                'catalog_product_id': catalog_id, 
                'item_id' : item.get('item_id'),
                'competitors_type': 'catalog',
                'category_id': item.get('category_id'),
                'official_store_id': item.get('official_store_id'),
                'competitor_seller_id': item.get('seller_id'),
                'listing_type_id': item.get('listing_type_id'),
                'condition': item.get('condition'),
            }
            
            results_list.append(dict_content)  # Append each dictionary to the list
        
        return results_list  # Return the full list after iterating through all items
    
    except Exception as e:
        print(f'Error processing json: {json}. Error: {str(e)}')
        return None  # Optionally return None if there's an error
